<a href="https://colab.research.google.com/github/Rish0x29/Python/blob/main/BusinessCase_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np

# We will use the sklearn preprocessing library, as it will be easier to standardize the data.
from sklearn import preprocessing

raw_csv_data = np.loadtxt('/content/sample_data/Audiobooks_data.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one [:,0]
# and the last one [:,-1] (which is our targets)

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [41]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

In [42]:
num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
indices_to_remove = []

# Count the number of targets that are 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [43]:
# It's a simple line of code, which standardizes the inputs, as we explained in one of the lectures.
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [44]:
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [45]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]


# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1779.0 3579 0.49706621961441744
219.0 447 0.4899328859060403
239.0 448 0.5334821428571429


In [46]:

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [47]:
import numpy as np
import tensorflow as tf

In [48]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

In [49]:
input_size = 10
output_size = 2
hidden_layer_size = 50

# define how the model will look like
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])



# the loss function,
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

batch_size = 100

max_epochs = 100

# set an early stopping mechanism
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2
          )

Epoch 1/100
36/36 - 1s - 33ms/step - accuracy: 0.6980 - loss: 0.5830 - val_accuracy: 0.7696 - val_loss: 0.4757
Epoch 2/100
36/36 - 0s - 4ms/step - accuracy: 0.7619 - loss: 0.4621 - val_accuracy: 0.7919 - val_loss: 0.4109
Epoch 3/100
36/36 - 0s - 5ms/step - accuracy: 0.7888 - loss: 0.4164 - val_accuracy: 0.7919 - val_loss: 0.3879
Epoch 4/100
36/36 - 0s - 5ms/step - accuracy: 0.7916 - loss: 0.3951 - val_accuracy: 0.8121 - val_loss: 0.3747
Epoch 5/100
36/36 - 0s - 6ms/step - accuracy: 0.7999 - loss: 0.3810 - val_accuracy: 0.8009 - val_loss: 0.3683
Epoch 6/100
36/36 - 0s - 3ms/step - accuracy: 0.8078 - loss: 0.3724 - val_accuracy: 0.8076 - val_loss: 0.3594
Epoch 7/100
36/36 - 0s - 3ms/step - accuracy: 0.8066 - loss: 0.3664 - val_accuracy: 0.8188 - val_loss: 0.3561
Epoch 8/100
36/36 - 0s - 3ms/step - accuracy: 0.8064 - loss: 0.3642 - val_accuracy: 0.8031 - val_loss: 0.3557
Epoch 9/100
36/36 - 0s - 3ms/step - accuracy: 0.8117 - loss: 0.3596 - val_accuracy: 0.7964 - val_loss: 0.3535
Epoch 10/

In [50]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7964 - loss: 0.3636 
